In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import numpy as np
import config, torch
from torch import nn
import os
from derma.dataset import Derma, get_samples_weight

In [ ]:
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, RandomVerticalFlip, RandomRotation
transform = Compose([
        RandomHorizontalFlip(), 
        RandomVerticalFlip(),
        RandomRotation(90),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

val_transform = Compose([
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

dataset = Derma(config.DATASET_DIR, transform=transform)
val_dataset = Derma(config.DATASET_DIR, transform=val_transform)

from torch.utils.data import DataLoader, random_split
val_size = int(0.1*len(dataset))
train_size = len(dataset) - val_size

train_set, val_set = random_split(dataset, [train_size, val_size])
val_set.dataset = val_dataset # Buscar una forma más elegante de hacerlo

train_sampler, _ = get_samples_weight(train_set)
train_loader = DataLoader(train_set, batch_size=64, shuffle=False, sampler=train_sampler)

val_sampler, _ = get_samples_weight(val_set)
val_loader = DataLoader(val_set, batch_size=512, shuffle=False, sampler=val_sampler)

In [ ]:
from derma.architecture import InvertedResidual
from torchvision.models import MobileNetV2

inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

# inverted_residual_setting = [
#         # t, c, n, s
#         [1, 16, 1, 1],
#         [2, 24, 1, 2],
#         [2, 32, 1, 2],
#         [2, 64, 1, 2],
#         [2, 96, 1, 1],
#         [2, 160, 1, 2],
#         [2, 320, 1, 1],
#     ]

labels = [0, 1]
model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting, block=InvertedResidual)
# model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting)

In [ ]:
from derma.utils import train
from torch.utils.tensorboard import SummaryWriter

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-6)
tb_writer = SummaryWriter(log_dir='test/CoordAtt')
criterion = torch.nn.CrossEntropyLoss()

train(model, [train_loader, val_loader], optimizer, criterion, 3, tb_writer)

In [ ]:
from torchvision.models import MobileNetV2
from derma.architecture import MobileNetDecoder

inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

encoder = MobileNetV2(inverted_residual_setting=inverted_residual_setting).features
decoder = MobileNetDecoder(inverted_residual_setting)
model = nn.Sequential(encoder, decoder)

In [ ]:
from derma.utils import train
from torch.utils.tensorboard import SummaryWriter

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
tb_writer = SummaryWriter(log_dir='reconstruction/')
criterion = nn.MSELoss()

train(model, [train_loader, val_loader], optimizer, criterion, 3, tb_writer, reconstruction=True)